In [34]:
import transformers
transformers.logging.set_verbosity_error()

In [30]:
import pandas as pd
from tqdm import tqdm

In [31]:
dev_df = pd.read_csv('data/input/dev.tsv', sep='\t')
toxic_inputs = dev_df['toxic_comment'].tolist()

## Library version

In [1]:
from russian_paraphrasers import GPTParaphraser

In [3]:
model_name = 'russian-paraphrasers-gpt3'

In [4]:
paraphraser = GPTParaphraser(model_name="gpt3", range_cand=True, make_eval=False)

In [5]:
dev_df = pd.read_csv('data/input/dev.tsv', sep='\t')
toxic_inputs = dev_df['toxic_comment'].tolist()

In [6]:
sentence = "Мерседес это самая быстрая машина на российском рынке."

In [ ]:
results = paraphraser.generate(
    sentence, n=10, temperature=1, 
    top_k=10, top_p=0.9, 
    max_length=100, repetition_penalty=1.5,
    threshold=0.7
)
results

In [26]:
results['results']

[{'predictions': ['"Мерс" признан самой быстрорастущей маркой в России.',
   'Самая быстрорастущая марка российского авто.',
   '"Меркава" названа самой быстрорастущей иномаркой в России.',
   'Названа максимальная скорость самой быстроизнашиваемой машины в России.',
   '"Меркава" названа самой быстрой машиной российского авторынка.',
   'Самая быстрорастущая марка российского автотранспорта.',
   'Самая быстрорастущая компания в России.',
   'Самая быстрорастущая марка автомобилей в России.',
   '" Мерседесе" принадлежит рекорд по скоростемеру.',
   'Мерседесе быстрее всех российских авто.'],
  'best_candidates': ['Самая быстрорастущая марка российского автотранспорта.',
   'Названа максимальная скорость самой быстроизнашиваемой машины в России.',
   '"Меркава" названа самой быстрой машиной российского авторынка.',
   'Самая быстрорастущая марка автомобилей в России.',
   'Самая быстрорастущая марка российского авто.',
   'Мерседесе быстрее всех российских авто.']}]

In [36]:
para_results = []
for i in tqdm(toxic_inputs):
    results = paraphraser.generate(
        i, n=10, temperature=1,
        top_k=10, top_p=0.9,
        max_length=100, repetition_penalty=1.5,
        threshold=0.7
    )
    para_result = []
    for result in results['results']:
        if result['best_candidates']:
            para_result.append(result['best_candidates'][-1])
        else:
            para_result.append(result['predictions'][0])
    para_results.append(' '.join(para_result))

100%|██████████| 800/800 [36:28<00:00,  2.74s/it]


In [42]:
with open(f'data/output/{model_name}_dev.txt', 'w', encoding='utf-8') as file:
    file.writelines([sentence+'\n' for sentence in para_results])

## HuggingFace version

In [17]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from russian_paraphrasers.utils import clean

In [3]:
checkpoint = 'alenusch/rugpt3-paraphraser'
model_name = checkpoint.split('/')[1]

In [4]:
model = GPT2LMHeadModel.from_pretrained(checkpoint)
tokenizer = GPT2Tokenizer.from_pretrained(checkpoint)
model.cuda();

In [5]:
sentence = 'Мама мыла раму'
prompt = f'<s>{sentence} === '

In [6]:
encoded_prompt = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
encoded_prompt = encoded_prompt.to(model.device)

In [7]:
encoded_prompt

tensor([[   32,    87,    34, 35695, 43068,   351,   597,   225,  6188,    33,
           225]], device='cuda:0')

In [6]:
input_ids = encoded_prompt

In [7]:
output_sequences = model.generate(
    input_ids=input_ids,
    max_length=100,
    temperature=1,
    top_k=10,
    top_p=0.9,
    repetition_penalty=1.5,
    do_sample=True,
    num_return_sequences=10
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [33]:
def paraphrase(
        text,
        model,
        tokenizer,
        max_length=100,
        temperature=1,
        top_k=10,
        top_p=0.9,
        repetition_penalty=1.0,
        do_sample=True,
        num_return_sequences=1,
        stop_token='</s>'
):
    prompt = f'<s>{text} === '
    encoded_prompt = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
    encoded_prompt = encoded_prompt.to(model.device)
    input_ids = encoded_prompt
    output_sequences = model.generate(
        input_ids=input_ids,
        max_length=max_length,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        repetition_penalty=repetition_penalty,
        do_sample=do_sample,
        num_return_sequences=num_return_sequences
    )
    output_text = tokenizer.decode(output_sequences[0])
    output_text = output_text[: output_text.find(stop_token) if stop_token else None]
    output_text = output_text[len(tokenizer.decode(encoded_prompt[0])):]
    output_text = clean(output_text)

    return output_text

In [35]:
para_results = []
for i in tqdm(toxic_inputs):
    para_result = paraphrase(i, model, tokenizer)
    para_results.append(para_result)

100%|██████████| 800/800 [19:33<00:00,  1.47s/it]


In [38]:
with open(f'data/output/{model_name}_dev.txt', 'w', encoding='utf-8') as file:
    file.writelines([sentence+'\n' for sentence in para_results])

In [39]:
model_name

'rugpt3-paraphraser'